In [18]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.linear_model import LinearRegression as model
from sklearn.metrics import mean_squared_error as loss
import pandas as pd
import seaborn as sns
sns.set(font_scale=1.3)
from scipy.stats import iqr


def get_bootstrap(data1, data2, iterations=1000, sign_level=0.05):
    mean_diff = np.median(data1) - np.median(data2)
    boot_data = []
    print(data1, data2)
    # Bootstrap sampling
    for i in range(iterations):
        curr_idxs1 = np.random.choice(len(data1), len(data1))
        curr_idxs2 = np.random.choice(len(data2), len(data2))
        boot_data.append(np.median(data1[curr_idxs1]) - np.median(data2[curr_idxs2]))
    
    boot_data = np.array(boot_data) - mean_diff
    # Calculate p-values
    p_value = sum(np.abs(boot_data) >= np.abs(mean_diff)) / len(boot_data)
    # print(mean_diff, boot_data)
    print(p_value)
    print(p_value > sign_level)

# import matplotlib.pylab as pylab

In [24]:
dict_ = np.load('dict_actualab.npy',allow_pickle='TRUE').item()
df = pd.DataFrame(dict_)
df.head()

,A,V,$\frac{a}{b}$,$$\frac{(A^2b^2 - a^2)(K-1)^2}{b^2(2K-1)}$$,A < a/b,V < V_cond,Learned - Real a,Learned - Real Ab,Method,Stable - Unstable Output,Stable Output,Unstable Output,Resulting Model MSE,% Select Stable Model
0,0.109414,0.095771,0.042454,0.038284,False,False,0.001379,0.749666,Random\nCross\nValidation,-0.097523,1.412185e-01,0.238742,0.000002,True
1,0.109414,0.095771,0.042454,0.038284,False,False,0.001379,0.749666,Block\nCross\nValidation,-0.129848,1.412183e-01,0.271067,0.000002,True
2,0.109414,0.095771,0.042454,0.038284,False,False,0.001379,0.749666,Proposed\nApproach,-0.032325,-1.902888e-07,0.032325,0.000002,True
3,0.206420,0.149291,0.036791,0.155316,False,True,-0.005410,1.010865,Random\nCross\nValidation,0.315020,1.946899e+00,1.631879,0.489532,False
4,0.206420,0.149291,0.036791,0.155316,False,True,-0.005410,1.010865,Block\nCross\nValidation,-0.060660,1.946968e+00,2.007628,0.000029,True


In [25]:
len(dict_['A'])/3

40000.0

In [26]:
df['A > a/b'] = df['A'] > df['$\\frac{a}{b}$']

In [27]:
df['V > V_cond'] = df['V'] > df['$$\\frac{(A^2b^2 - a^2)(K-1)^2}{b^2(2K-1)}$$']

In [28]:
import math
df_curr = df[(df['A > a/b']) & (df['V < V_cond'])]
rand_spec = df_curr[df_curr['Method'] == 'Random\nCross\nValidation']['Resulting Model MSE'].quantile([0.25, 0.5,0.75]).round(4)
block_spec = df_curr[df_curr['Method'] == 'Block\nCross\nValidation']['Resulting Model MSE'].quantile([0.25, 0.5,0.75]).round(4)
prop_spec = df_curr[df_curr['Method'] == 'Proposed\nApproach']['Resulting Model MSE'].quantile([0.25, 0.5,0.75]).round(4)


text1 = '%0.6f, 50\%% PI: [%0.6f, %0.6f]\n'%(prop_spec[0.5], prop_spec[0.25], prop_spec[0.75])
text1 += '(%0.6f, 50\%% PI: [%0.6f, %0.6f])\n'%(rand_spec[0.5], rand_spec[0.25], rand_spec[0.75])
text1 += '(%0.6f, 50\%% PI: [%0.6f, %0.6f])'%(block_spec[0.5], block_spec[0.25], block_spec[0.75])
print(text1)

0.000000, 50\% PI: [0.000000, 0.000300]
(3.185400, 50\% PI: [0.000000, 32.745500])
(2.343000, 50\% PI: [0.000000, 32.189300])


In [29]:
get_bootstrap(df_curr[df_curr['Method'] == 'Proposed\nApproach']['Resulting Model MSE'].to_numpy(), df_curr[df_curr['Method'] == 'Random\nCross\nValidation']['Resulting Model MSE'].to_numpy())
get_bootstrap(df_curr[df_curr['Method'] == 'Proposed\nApproach']['Resulting Model MSE'].to_numpy(), df_curr[df_curr['Method'] == 'Block\nCross\nValidation']['Resulting Model MSE'].to_numpy())

[2.88435290e-05 4.53655838e-05 2.96541443e-05 ... 1.17168464e-05
 1.01650469e-05 1.87657493e-05] [4.89532015e-01 3.06270370e-01 2.24231204e-01 ... 1.17168464e-05
 1.01650469e-05 1.87657493e-05]
0.0
False
[2.88435290e-05 4.53655838e-05 2.96541443e-05 ... 1.17168464e-05
 1.01650469e-05 1.87657493e-05] [2.88435290e-05 3.06270370e-01 2.24231204e-01 ... 1.17168464e-05
 1.01650469e-05 1.87657493e-05]
0.0
False


In [30]:
import math
df_curr = df[(df['A < a/b']) | (df['V > V_cond'])]
rand_spec = df_curr[df_curr['Method'] == 'Random\nCross\nValidation']['Resulting Model MSE'].quantile([0.25, 0.5,0.75]).round(4)
block_spec = df_curr[df_curr['Method'] == 'Block\nCross\nValidation']['Resulting Model MSE'].quantile([0.25, 0.5,0.75]).round(4)
prop_spec = df_curr[df_curr['Method'] == 'Proposed\nApproach']['Resulting Model MSE'].quantile([0.25, 0.5,0.75]).round(4)


text1 = '%0.6f, 50\%% PI: [%0.6f, %0.6f]\n'%(prop_spec[0.5], prop_spec[0.25], prop_spec[0.75])
text1 += '(%0.6f, 50\%% PI: [%0.6f, %0.6f])\n'%(rand_spec[0.5], rand_spec[0.25], rand_spec[0.75])
text1 += '(%0.6f, 50\%% PI: [%0.6f, %0.6f])'%(block_spec[0.5], block_spec[0.25], block_spec[0.75])
print(text1)

0.000100, 50\% PI: [0.000000, 0.000800]
(0.000100, 50\% PI: [0.000000, 0.001000])
(0.000100, 50\% PI: [0.000000, 0.000800])


In [31]:
get_bootstrap(df_curr[df_curr['Method'] == 'Proposed\nApproach']['Resulting Model MSE'].to_numpy(), df_curr[df_curr['Method'] == 'Random\nCross\nValidation']['Resulting Model MSE'].to_numpy())
get_bootstrap(df_curr[df_curr['Method'] == 'Proposed\nApproach']['Resulting Model MSE'].to_numpy(), df_curr[df_curr['Method'] == 'Block\nCross\nValidation']['Resulting Model MSE'].to_numpy())

[1.92699995e-06 3.75787387e-09 1.27640562e-05 ... 7.45940842e-04
 1.32694701e-04 4.42013450e-07] [1.92699995e-06 3.75787387e-09 1.27640562e-05 ... 7.45940842e-04
 1.32694701e-04 4.42013450e-07]
0.009
False
[1.92699995e-06 3.75787387e-09 1.27640562e-05 ... 7.45940842e-04
 1.32694701e-04 4.42013450e-07] [1.92699995e-06 3.75787387e-09 1.27640562e-05 ... 7.45940842e-04
 1.32694701e-04 4.42013450e-07]
0.662
True
